### Example For Calculation of Self Solvation Free Energy

#### Do Imports

In [ ]:
from rdkit import Chem
from pygromos.files.gromos_system.ff.forcefield_system import forcefield_system
from pygromos.simulations.approaches.solvation_free_energy_calculation.solvation_free_energy import Solvation_free_energy_calculation

#### Choose Molecule to run calculation for

In [ ]:
smiles = "c1ccccc1"

#### Calculate or set the number of atoms

In [ ]:
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
number_of_atoms = mol.GetNumAtoms()

#### Create The Solvation free energy calculation system

In [ ]:
workfolder = "/home/kpaul/pygromos_new_fork/PyGromosTools/testbench3"
sf = Solvation_free_energy_calculation(input_system=smiles, # Gromos_System, SMILES (str) or rdkit Mol
                                       work_folder=workfolder, # Folder to do calculations in
                                       system_name=smiles, # Name of the system (does not need to be smiles but convenient)
                                       forcefield=forcefield_system("openforcefield"), # Force field to use
                                       density=789, # density of the liquid in kg/L
                                       num_molecules=512, # number of molecules used for the calculation
                                       num_atoms=number_of_atoms, # number of atoms in one molecule
                                       nmpi=1, nomp=1, # number of mpi and omp cores
                                       subsystem="local", # Subsystem to use for calculation local or lsf
                                       amberscaling=True) # Whether to use amberscaling (for openforcefield recommended)

#### Create Liquid

In [ ]:
sf.create_liq()

#### Minimize Liquid

In [ ]:
emin_sys, jobID = sf.minimize_liq(in_gromos_simulation_system=sf.groSys_liq,prev_JobID=-1)

#### Change the number of cores for longer runs

In [ ]:
sf.nmpi = 8

#### Equilibrate System

In [ ]:
eq_sys, jobID = sf.eq_liq(in_gromos_simulation_system=emin_sys,prev_JobID=jobID)

#### Do Ti calculation

In [ ]:
ti_sys, jobID = sf.ti_liq(in_gromos_simulation_system=eq_sys,
                          prev_JobID=jobID,
                          n_points=5) # Number of Lambda points to calculate (typically 21)

#### Read out results

In [ ]:
sf.calculate_solvation_free_energy(n_points=5) # Number of Lambda points to use for calculation (typically 21)